# Least Squares Linear Regression

In this lecture we will introduce **linear models** and **least squares linear regression**.  We will explain the basic *parametric model* formulation and how to optimize it using **linear algebra** and **geometry** rather than gradient descent.  

**Updated** This notebook has been slightly updated with additional explanations from the original posting.  I have also added a video walk-through of the notebook which is imported at the very end.

## Imports 

In this notebook we will be using the following packages:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import cufflinks as cf
cf.set_config_file(offline=True, sharing=False, theme='ggplot');

In [ ]:
# Support embedding YouTube Videos in Notebooks
from IPython.display import YouTubeVideo

## A Note on Lecture Format

This is the first of a series of lectures that are designed to be completed online rather than in class.  This new lecture format is experimental and we ask for your feedback on Piazza. 

The playlist for all the videos in this lecture can be found here:

[Least Squares Linear Regression](https://www.youtube.com/playlist?list=PLcK2S75CXo8MNPMb80AZOfwCFfWIdQFPV)

### Intended Use
We have designed this video notebook to be a series of short videos followed by simple questions (with answers provided) and demos. There are no grades for working through this notebook.  However, we hope that by combining videos with activities this will actually be more engaging.

## Recap

The following short video provides a recap of the modeling recipe (model, loss, optimize).  If you are comfortable with this you can safely skip this video.

In [ ]:
YouTubeVideo("u0YycwQ814c")

### Recap Question

What are the three stages of modeling used in this class?

<details>
    <summary>Answer</summary>
    In this class, the recipe for modeling consists of defining the model, defining the loss, and optimizing the model with respect to the loss.
</details>

## Multilinear Regression Setting

In the following video we introduce the multiple linear regression setting. 

In [ ]:
YouTubeVideo("21PN6a3s22I")

## The Diamonds Dataset

To motivate the multilinear regression setting we will work with the classic diamonds dataset. 

In [ ]:
data = pd.read_csv("data/diamonds.csv.zip", index_col=0)
data.head()

Each record in the dataset corresponds to a single diamond.  The fields are

1. **carat**: The weight of the diamonds
2. **cut**: The quality of the cut is an *ordinal* variable with values (`Fair`, `Good`, `Very Good`, `Premium`, and `Ideal`)
3. **color**: The color of the diamond is an *ordinal* variable with values `J` (worst) to `D` (best).
4. **clarity**: How obvious inclusions are within the diamond. This is an *ordinal* variable with values `I1` (worstt), `SI2`, `SI1`, `VS2`, `VS1`, `VVS2`, `VVS1`, `IF` (best)
5. **depth**: The height of a diamond, measured from the culet to the table, divided by its average girdle diameter.
6. **table**: The width of the diamond's table expressed as a percentage of its average diameter.
7. **price**: Price of the diamond in USD.
8. **x**: length measured in mm
9. **y**: width measured in mm
10. **z**: depth measured in mm

We are interested in **predicting the price of a diamond given it's characteristics**.  Therefore, we would like to fit a model:

$$
f_\theta\left(\text{Diamond's Characteristics}\right) \rightarrow \text{Price}
$$

Suppose for now we focus only on the diamond's **carat**, **depth**, and **table** values.  In this case, we would want a model of the form:

$$
f_\theta\left(\textbf{carat}, \textbf{depth}, \textbf{table}\right) \rightarrow \textbf{price}
$$

We could express this as a linear model of the form:

$$
f_\theta\left(\textbf{carat}, \textbf{depth}, \textbf{table}\right) 
=
\theta_0 + 
\theta_1 * \textbf{carat} +
\theta_2 * \textbf{depth} +
\theta_3 * \textbf{table} 
$$

Where $\theta_0$ is the **intercept** parameter and $\theta_1$, ..., $\theta_3$ are the **"slopes"** with respect to each of the covariates.  

Focusing on just the **carat**, **depth**, and **table** features and the price our dataset looks like:

In [ ]:
data[['carat', 'depth', 'table', 'price']]

In lecture, we saw that the predictions for the entire data set $\hat{\mathbb{Y}}$ can be computed as:

$$
\hat{\mathbb{Y}} = \mathbb{X} \theta  
$$

The **covariate matrix** $\mathbb{X} \in \mathbb{R}^{n,d+1}$ has $n$ rows corresponding to each record in the dataset and $d+1$ columns corresponding to the original $d$ columns in the data plus an additional 1s column vector.  For example, the first five rows of $\mathbb{X}$ for this dataset would look like:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1.0</td>
      <td>0.23</td>
      <td>61.5</td>
      <td>55.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1.0</td>
      <td>0.21</td>
      <td>59.8</td>
      <td>61.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1.0</td>
      <td>0.23</td>
      <td>56.9</td>
      <td>65.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1.0</td>
      <td>0.29</td>
      <td>62.4</td>
      <td>58.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1.0</td>
      <td>0.31</td>
      <td>63.3</td>
      <td>58.0</td>
    </tr>
  </tbody>
</table>


We can extract our covariate matrix and add a column of all 1s:

In [ ]:
n,d = data[['carat', 'depth', 'table']].shape
print("n = ", n)
print("d = ", d)

The following function attaches 1s to each of the rows in the matrix.  Here I am using the `np.hstack` function to add an extra column.

In [ ]:
def add_ones_column(data):
    return np.hstack([np.ones((n,1)), data])

We can then add a 1s column to our dataset.

In [ ]:
X = add_ones_column(data[['carat', 'depth', 'table']].to_numpy())
X

We can define the linear model in python using numpy.  Here I will assume the input is in row vector form so this function will work with a single row or the entire matrix.  Note in lecture we discuss the case where a single row is stored in column vector form (as this is the case in most math textbooks).

The `@` symbol is the matrix multiply operation and is equivalent to writing `xt.dot(theta)`.

In [ ]:
def linear_model(theta, xt): 
    return xt @ theta # The @ symbol is matrix multiply

For any value of $\theta$ (even a random one) we can make a prediction using our model:

In [ ]:
np.random.seed(42)
theta_guess = np.random.randn(d+1, 1)
theta_guess

_Note: Above, we used `np.random.randn`, which generates draws from a normal (Gaussian) distribution with mean 0 and variance 1 (the parameters `d+1` and `1` specify the number of rows and columns that the returning array should have). The fact that these values are drawn from a normal distribution are irrelevant here, just think of them as coming from a random number generator._

Making a single prection for the 3rd row:

In [ ]:
linear_model(theta_guess, X[2,:])

Making a prediction for all the rows at once:

In [ ]:
linear_model(theta_guess, X)

Copying the notation directly from the video lecture:

In [ ]:
Y_hat = X @ theta_guess
Y_hat

or using our function produces the same thing:

In [ ]:
Y_hat = linear_model(theta_guess, X)
Y_hat

## Defining and Minimizing the Loss

The next video defines and minimizes the least squares loss by using a geometric argument.  

In [ ]:
YouTubeVideo("zkJ3CULN8Wk")

To elaborate a little on the video above:

Recall from linear algebra that two vectors are orthogonal when their dot product is zero. The standard way to write the dot product is $a \cdot b$, but it can also be written as $a^T b$ or $b^T a$ (all of these forms evaluate to $a_1 b_1 + a_2 b_2 + ... + a_n b_n$).

The notion of orthogonality extends to matrices. If we want some vector $r$ to be orthogonal to the span of a matrix $X$ (i.e. to each column in the matrix), that's the same as saying the vector is orthogonal to the entire matrix. To denote this, we can say $X \cdot r = 0$ or $X^T \cdot r = 0$. In our case, $r = y - X  \hat{\theta}$, so we have $X^T (y - X \hat{\theta}) = 0$.

## The Least Squares Loss

The Least Squares Regression loss is the average squared loss:

\begin{align}
L(\theta) &= \frac{1}{n}\sum_{i=1}^n \left( \mathbb{Y}_i - \left(\mathbb{X} \theta\right)_i \right)^2 \\
&= \frac{1}{n}\sum_{i=1}^n \left( \mathbb{Y}_i - \mathbb{X}_i \theta \right)^2 \\
&= \frac{1}{n} || \mathbb{Y} - \mathbb{X} \theta ||_2^2 \\
&= \frac{1}{n}\left( \mathbb{Y} - \mathbb{X}\theta \right)^T \left( \mathbb{Y} - \mathbb{X}\theta \right)
\end{align}

Note: Above, we used 2 facts from linear algebra. First, we used the fact that the square of the $L_2$ norm of a vector is equal to the sum of the squares of its components. Put less cryptically, that means that for any vector $r$, $||r||_2^2 = r_1^2 + r_2^2 + ... + r_n^2 = \sum_{i = 1}^n r_i^2$. We used this going from line 2 to 3, where $r = \mathbb{Y} - \mathbb{X} \theta$. The second fact we used was that (again for any vector $r$) $||r||_2^2 = r \cdot r = r^Tr$.

Here $\mathbb{Y}_i$ is the $i$th observed response (diamond price) and $\mathbb{X}_i$ is the **row vector** coresponding to the $i$th row of the covariates with an added 1 at the beginning. 

It is worth noting that since our goal is typically to minimize the loss and we often don't care about its actual value we can drop the $\frac{1}{n}$:


\begin{align}
L(\theta) &= \left( \mathbb{Y} - \mathbb{X}\theta \right)^T \left( \mathbb{Y} - \mathbb{X}\theta \right)
\end{align}


### Calculating the Loss

In order to calculate the loss we need the response vector $\mathbb{Y}$ (the thing we are trying to predict).

In [ ]:
Y =  data[['price']].to_numpy()
Y

Directly writing the **average squared loss**:

In [ ]:
def squared_loss(theta):
    return np.mean((Y - X @ theta)**2)

In [ ]:
squared_loss(theta_guess)

Using matrix notation:

In [ ]:
def squared_loss(theta):
    return ((Y - X @ theta).T @ (Y - X @ theta)).item() / Y.shape[0]

In [ ]:
squared_loss(theta_guess)

You may notice that there is some variation in the last digits of the floating point calculation.  This is an issue with ordering of numerical operations over large sums.  

## Minimizing the Loss

In the video lecture we defined the normal equation as:

$$
\mathbb{X}^T \mathbb{X} \hat{\theta} = \mathbb{X}^T \mathbb{Y}
$$

If we solve for $\hat{\theta}$ this is the minimizer of the squared loss with respect to our data.  

If we assume $\mathbb{X}^T \mathbb{X}$ is invertible (full rank) then we get the following solution:

$$
 \hat{\theta} = \left( \mathbb{X}^T \mathbb{X} \right)^{-1} \mathbb{X}^T \mathbb{Y}
$$


## Understanding the Normal Equation

The following video provides a better intuition of the shape of each of the terms in the solution to the normal equation as well as how to better solve it.

In [ ]:
YouTubeVideo("9vWR-19WQKU")

## Implementing the Normal Equation

While it is not as numerically stable or efficient. We can compute $\hat{\theta}$ by direction using matrix inversion.  To do this, we import the `inv` function for the numpy linear algebra library:

In [ ]:
from numpy.linalg import inv

In [ ]:
theta_hat = inv(X.T @ X) @ X.T @ Y
theta_hat

A more efficient way to solve the normal equations is using the `solve` function to solve the linear systems:

$$
A \theta = b
$$

where $A=\mathbb{X}^T \mathbb{X}$ and $b=\mathbb{X}^T \mathbb{Y}$.  The `solve` function can be implement slightly more efficiently by avoiding the inversion and subsequent matrix multiply.  For matrices which are less well behaved, solve can also be more numerically stable. 

In [ ]:
from numpy.linalg import solve

In [ ]:
theta_hat = solve(X.T @ X, X.T @ Y)
theta_hat

We have just computed the global optimum of the squared loss.  

## Making Predictions

Now that we have estimated the model parameters $\hat{\theta}$ we can now also predict the price $\hat{\mathbb{Y}}$ for each of our diamonds

In [ ]:
Y_hat = X @ theta_hat

### Diagnosing the Model

In previous lectures we have plotted the residual.  We can do that for each of the dimensions but in general it will be difficult to visualize the residual directly.

#### Residual vs Carat

In [ ]:
fig = px.scatter(x = X[:,1], y = Y - Y_hat,opacity=0.2)
fig.update_xaxes(title="Carat")
fig.update_yaxes(title="Residual")
fig

From the above plot we see that we are over estimating the price of big diamonds and under estimating the price of small diamonds.

#### Residual vs Depth

In [ ]:
fig = px.scatter(x = X[:,2], y = Y - Y_hat,opacity=0.2)
fig.update_xaxes(title="Depth")
fig.update_yaxes(title="Residual")
fig

There is no consistent pattern in the residuals.

#### Residuals vs Table

In [ ]:
fig = px.scatter(x = X[:,3], y = Y - Y_hat,opacity=0.2)
fig.update_xaxes(title="Table")
fig.update_yaxes(title="Residual")
fig

### Plotting $\hat{\mathbb{Y}}$ vs $\mathbb{Y}$
A better way to visualize higher dimensional regression models is to compare the Predicted Value against the Observed value

In [ ]:
fig = px.scatter(x = Y.flatten(), y = Y_hat.flatten(), opacity=0.2)
fig.add_trace(go.Scatter(x=[0,20000], y=[0, 20000], name="Y_hat=Y"))
fig.update_xaxes(title="Y")
fig.update_yaxes(title="Y_hat")
fig

What do we see?  We are under estimating expensive diamonds!  Perhaps we should consider the cut, color, and clarity.  

# Video Walk-Through

Several of you asked for a video walk-through of the notebook.  I have created one here:

In [ ]:
YouTubeVideo("7M3yLv_DYX4")

## Whats Next

In the next notebook lecture we will see how to improve this model by applying feature engineering techniques to incorporate categorical data.